In [ ]:
# 開啟LINE BOT 起始點
!pip install Flask 
!pip install flask pyngrok
!pip install passlib
!pip install flask-cors
!pip install pandas
!pip install sqlalchemy pymysql
!pip install line-bot-sdk --upgrade

In [ ]:
import requests
import hashlib
from flask import Flask, request, abort, render_template, jsonify  # 確保導入 render_template
from pyngrok import ngrok
from linebot import LineBotApi, WebhookHandler
import pymysql
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from linebot.exceptions import InvalidSignatureError
from linebot.models import TextSendMessage, TemplateSendMessage, ButtonsTemplate, MessageAction, MessageEvent, TextMessage, URIAction, PostbackAction, FlexSendMessage, BubbleContainer, TextComponent, BoxComponent, SeparatorComponent
from datetime import datetime
from flask_cors import CORS

# 初始化 Flask 應用和 ngrok
app = Flask(__name__)
CORS(app)
port = 5000  # 設定 Flask 使用的端口

# 使用你的 ngrok authtoken
ngrok.set_auth_token('XXXX')

# 建立 ngrok 隧道
public_url = ngrok.connect(port).public_url
print(f"ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

line_bot_api = LineBotApi('XXXX')
handler = WebhookHandler('XXXX')

# 設定 OpenAI API 金鑰
openai_api_key = "XXXX"

@app.route("/", methods=['POST'])
def callback():
    # 從請求中獲取 X-Line-Signature 頭部和請求體
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 設置 MySQL 連接參數
db_settings = {
    'host': '127.0.0.1',
    'port': 3306,
    'user': 'OUTS',
    'password': 'test12345',
    'db': 'balance_diet'
}
engine = create_engine(f"mysql+pymysql://{db_settings['user']}:{db_settings['password']}@{db_settings['host']}:{db_settings['port']}/{db_settings['db']}")
Session = sessionmaker(bind=engine)
# 開始：推薦功能
#推薦
@app.route("/Recommend")
def Recommend():
    # 每日建議攝取值
    daily_nutrients = {
        'Energy': 2400,
        'Protein': 90,
        'Total_lipid_fat': 60,
        'Carbohydrate': 345
    }
    # 產生菜單
    menu, meal_nutrient_totals, nutrient_totals, meal_nutrient_limits = generate_menu(daily_nutrients)
    print(menu)
    # 將每一餐的菜單分別賦值
    breakfast_menu = menu.get("早餐", [])
    lunch_menu = menu.get("午餐", [])
    dinner_menu = menu.get("晚餐", [])
    beverage_menu = menu.get("飲料", [])
    snack_menu = menu.get("點心", [])
    
    return render_template("Recommend.html", standard=daily_nutrients, 
                           breakfast_menu=breakfast_menu, lunch_menu=lunch_menu, dinner_menu=dinner_menu, 
                           beverage_menu=beverage_menu, snack_menu=snack_menu)
    
# 路由 - 返回菜單推薦的數據
@app.route('/api/food')
def get_food_data():
    food_data = fetch_food_data()
    return jsonify(food_data)  # 將數據返回給前端
# 結束：推薦功能

#登入    
@app.route("/Login")
def Login():
    return render_template("Login.html")

#註冊   
@app.route("/Register")
def Register():
    return render_template("Register.html")

    
# 日誌
@app.route("/View")
def View():
    return render_template("View.html")

# 新增日期區間處理 API
@app.route("/submit-dates", methods=["POST"])
def submit_dates():
    data = request.json
    start_date = data.get('start_date')
    end_date = data.get('end_date')
    line_user_id = data.get('line_user_id')
    # 日期格式檢查與處理
    try:
        start = datetime.fromisoformat(start_date).strftime("%Y-%m-%d 00:00:00")
        end = datetime.fromisoformat(end_date).strftime("%Y-%m-%d 23:59:59")
        print("start",start)
        print("end",end)
        if start > end:
            return jsonify({"error": "開始日期不能晚於結束日期"}), 400

        if not line_user_id:
            return jsonify({'status': 'fail', 'message': 'LINE 使用者 ID 不得為空'}), 400

        session = Session()

        # 查詢-SQL結果1：使用者建議攝取數據
        query1 = text("""
            SELECT 
                user.id, user.name,
                dris.energy, dris.protein_min, dris.protein_max,
                dris.carbohydrate_min, dris.carbohydrate_max,
                dris.lipid_fat_min, dris.lipid_fat_max,
                six.grains_num, six.meat_beans_num, six.milk_num,
                six.vegetables_num, six.fruits_num, six.oil_num
            FROM user
            JOIN dris_standards AS dris ON user.standards_id = dris.id
            JOIN six_type_need AS six ON dris.six_type_need_id = six.id
            WHERE user.id = :line_user_id;
        """)
        try:
            user_data = session.execute(query1, {"line_user_id": line_user_id}).mappings().fetchone()
            user_data = dict(user_data) if user_data else None
        except Exception as e:
            print("查詢 SQL 結果1 失敗:", str(e))
            return jsonify({"error": "查詢 SQL 結果1 失敗"}), 500

        # 查詢-SQL結果2：飲食紀錄數據
        query2 = text("""
            SELECT 
                user_id, input_text, meal_type, energy, protein,
                carbohydrate, lipid_fat, grains, meat_beans, milk,
                vegetables, fruits, oil_nuts, create_time
            FROM record 
            WHERE user_id = :line_user_id
              AND create_time BETWEEN :start_date AND :end_date
            ORDER BY create_time DESC;
        """)
        try:
            record_data = session.execute(
                query2,
                {"line_user_id": line_user_id, "start_date": start, "end_date": end}
            ).mappings().fetchall()
            record_data = [dict(record) for record in record_data]
        except Exception as e:
            print("查詢 SQL 結果2 失敗:", str(e))
            return jsonify({"error": "查詢 SQL 結果2 失敗"}), 500

        # 組合-回傳清單1：計算日平均攝取數據與三大類營養素占比
        daily_energy = {}
        total_energy = total_protein = total_carbohydrate = total_lipid_fat = 0
        for record in record_data:
            date = record['create_time'].strftime('%Y-%m-%d')
            if date not in daily_energy:
                daily_energy[date] = 0
            daily_energy[date] += record['energy']
            total_energy += record['energy']
            total_protein += record['protein']
            total_carbohydrate += record['carbohydrate']
            total_lipid_fat += record['lipid_fat']

        num_days = len(daily_energy)
        avg_energy = round(total_energy / num_days, 2) if num_days > 0 else 0
        avg_protein = round(total_protein / num_days, 2) if num_days > 0 else 0
        avg_carbohydrate = round(total_carbohydrate / num_days, 2) if num_days > 0 else 0
        avg_lipid_fat = round(total_lipid_fat / num_days, 2) if num_days > 0 else 0

        if avg_energy > 0:
            protein_pct = round((avg_protein * 4 / avg_energy) * 100)
            carbohydrate_pct = round((avg_carbohydrate * 4 / avg_energy) * 100)
            lipid_fat_pct = round((avg_lipid_fat * 9 / avg_energy) * 100)
        else:
            protein_pct = carbohydrate_pct = lipid_fat_pct = 0

        nutrients_list = [
            avg_energy, avg_protein, avg_carbohydrate, avg_lipid_fat,
            protein_pct, lipid_fat_pct, carbohydrate_pct,            
            user_data['energy'], user_data['protein_min'], user_data['protein_max'],
            user_data['carbohydrate_min'], user_data['carbohydrate_max'],
            user_data['lipid_fat_min'], user_data['lipid_fat_max']
        ]

        # 組合-回傳清單2：六大類食物圖表資料
        suggested_foods = [
            user_data['grains_num'], user_data['meat_beans_num'],
            user_data['milk_num'], user_data['vegetables_num'],
            user_data['fruits_num'], user_data['oil_num']
        ]
        
        # 計算六大類食物每日平均值
        daily_foods = {}  # 用於存儲每天的六大類食物數據
        total_foods = [0, 0, 0, 0, 0, 0]  # 累計六大類食物數據
        for record in record_data:
            date = record['create_time'].strftime('%Y-%m-%d')
            if date not in daily_foods:
                daily_foods[date] = [0, 0, 0, 0, 0, 0]
            # 累加每一天的食物數據
            daily_foods[date][0] += record['grains']
            daily_foods[date][1] += record['meat_beans']
            daily_foods[date][2] += record['milk']
            daily_foods[date][3] += record['vegetables']
            daily_foods[date][4] += record['fruits']
            daily_foods[date][5] += record['oil_nuts']
        # 計算每日平均值
        num_days = len(daily_foods)
        avg_foods = [round(sum(values) / num_days, 2) if num_days > 0 else 0 for values in zip(*daily_foods.values())]
        
        food_list = []
        for i in range(6):
            food_list.extend([suggested_foods[i], avg_foods[i]])

        # 組合-回傳清單3：每日飲食紀錄
        meal_type_map = {'0': "早餐", '1': "午餐", '2': "晚餐", '3': "點心"}
        records_list = [
            {
                "date": record['create_time'].strftime('%Y-%m-%d'),
                "meal_type": meal_type_map.get(record['meal_type'], "未知"),
                "input_text": record['input_text']
            }
            for record in record_data
        ]

        result = {
            "nutrients_list": nutrients_list,
            "food_list": food_list,
            "records_list": records_list
        }
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    finally:
        session.close()

@app.route("/submit-login", methods=["POST"])
def submit_login():
    username = request.form['username']
    password = request.form['password']
    line_user_id = request.form.get('line_user_id')  # 從表單中取得 LINE 使用者的 ID
    print(f"使用者輸入的手機號碼: {username}")
    print(f"使用者輸入的密碼: {password}")
    #return f"手機號碼: {username}, 密碼: {password}"

    # 調試：檢查 line_user_id 是否正確
    print(f"LINE 使用者 ID: {line_user_id}")

    if not line_user_id:
        return jsonify({'status': 'fail', 'message': 'LINE 使用者 ID 不得為空'}), 400
    # 發送「[帳號] 登入成功」訊息
    message = TextSendMessage(text=f'{username} 登入成功')
    line_bot_api.push_message(line_user_id, message)

    return jsonify({'status': 'success'})

@app.route('/submit-register', methods=['POST'])
def submit_register():
    # 打印接收到的表單數據，檢查是否收到正確的數據
    print(request.form)

    # 確保從 request.form 來獲取數據，而不是 request.json
    phone = request.form.get('phone')
    email = request.form.get('email')
    confirm_password = request.form.get('confirmPassword')
    realname = request.form.get('realname')
    id_number = request.form.get('idNumber')
    gender = request.form.get('gender')
    birthdate = request.form.get('birthdate')
    height = request.form.get('height')
    weight = request.form.get('weight')
    activity_level = request.form.get('activityLevel')
    allergens = request.form.get('allergens')
    preferences = request.form.get('preferences')

    # 返回 JSON 響應
    return jsonify({'status': 'success', 'message': '註冊資料已收到並打印到控制台'})

   
# 用來追踪使用者的狀態
'''
user_state = {
    "update_data": None,  # 追踪使用者正在更新的項目（身高或體重）
    #"logged_in": False      # 追踪使用者是否已登入
}
'''
# 用來追踪使用者的狀態
user_state = {}

# 預設使用者的身高和體重
user_data = {
    "height": 160,
    "weight": 50,
    "allergens":{"乳製品","魚類"},
    "preferences":{"麵","起司"}
}

# 初始化一個變數來追蹤使用者是否正在記錄飲食
recording_food = {}
# 測試暫存USERID
TEMP_USERID = ""
# 處理使用者發送的文字訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    UserId = event.source.user_id
    text = event.message.text.strip().lower()

    global recording_food  # 使用全域變數來儲存是否正在記錄飲食

    # 根據文字內容處理不同邏輯
    if text == "登入":
        send_login_template(event.reply_token, UserId)
    elif text == "更新數值":
        send_update_template(event.reply_token, UserId)
    elif text in ["更新身高", "更新體重", "更新過敏原", "更新飲食偏好"]:  # 如果使用user_state者選擇了更新數值種類
        handle_update_data(event, UserId, text)
    elif text == "推薦菜單":
        send_recommend_template(event.reply_token, UserId)
    elif text == "飲食日誌":
        send_view_template(event.reply_token, UserId)
    elif text == "記錄飲食":
        send_food_template(event.reply_token, UserId)
    elif text in ["記錄早餐", "記錄午餐", "記錄晚餐", "記錄點心"]:  # 如果使用者選擇了餐別
        # 啟動記錄飲食模式
        recording_food[UserId] = {"recording": True}
        print("進入前recording_food[UserId]",recording_food[UserId])
        handle_food_input(event, UserId, text)  # 處理具體的飲食紀錄
    elif recording_food.get(UserId):  # 如果正在記錄飲食
        # 使用者輸入的下一條訊息應該是具體的食物內容
        handle_food_record(event)
        # 完成記錄後將狀態重置
        #recording_food[UserId] = False
    elif user_state.get(UserId, {}).get("update_type"):  # 判斷是否正在更新數值
        process_update(event, text)  # 處理使用者輸入的新數值
    else:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="不好意思，我不明白你的意思。"))


def send_login_template(reply_token, line_user_id):
    buttons_template = TemplateSendMessage(
        alt_text='登入和註冊',
        template=ButtonsTemplate(
            title='歡迎登入',
            text = '點擊下方按鈕綁定平台帳號吧～\n(如未有平台帳號請先註冊)',
            actions=[  # 使用 actions，而非 URIActions
                URIAction(label='綁定帳號', uri='https://liff.line.me/2006163998-1N4LxpBk'), # 每個按鈕都是一個 URIAction 物件
                URIAction(label='註冊帳號', uri='https://liff.line.me/2006163998-E8dYkX1P')# 每個按鈕都是一個 URIAction 物件
            ]
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)
    

# 傳送更新數值選項模板
def send_update_template(reply_token, line_user_id):
    send_update_message = TemplateSendMessage(
        alt_text='更新數值選單',
        template=ButtonsTemplate(
            title='更新數值',
            text='請選擇你要更新的數值：',
            actions=[
                MessageAction(label='身高', text='更新身高'),
                MessageAction(label='體重', text='更新體重'),
                MessageAction(label='過敏原', text='更新過敏原'),
                MessageAction(label='飲食偏好', text='更新飲食偏好')
            ]
        )
    )
    line_bot_api.reply_message(reply_token, send_update_message)

# 更新資料的邏輯：依據不同類型執行對應處理
def handle_update_data(event, UserId, update_type):
    if update_type == "更新身高":
        send_height_update_prompt(event)
    elif update_type == "更新體重":
        send_weight_update_prompt(event)
    elif update_type == "更新過敏原":
        send_allergen_update_prompt(event)
    elif update_type == "更新飲食偏好":
        send_preference_update_prompt(event)

    # 儲存狀態以便後續處理
    user_state[UserId] = {"update_type": update_type}

# 各類型的更新提示訊息
def send_height_update_prompt(event):
    current_value = user_data["height"]
    main_text = f"您目前的身高為 {current_value} 公分，"
    small_text = "請輸入您的新身高：\n(輸入數字即可，EX：150)"
    bubble = create_flex_message(main_text, small_text)
    line_bot_api.reply_message(event.reply_token, bubble)

def send_weight_update_prompt(event):
    current_value = user_data["weight"]
    main_text = f"您目前的體重為 {current_value} 公斤，"
    small_text = "請輸入您的新體重：\n(輸入數字即可，EX：50)"
    bubble = create_flex_message(main_text, small_text)
    line_bot_api.reply_message(event.reply_token, bubble)

def send_allergen_update_prompt(event):
    current_value = ",".join(user_data["allergens"])  # 使用 join 將集合轉換為逗號分隔的字串
    main_text = f"您目前的過敏原為 : {current_value} "  
    small_text = "請輸入新增的過敏原：(輸入文字即可，EX：花生)"  # 合併字串
    bubble = create_flex_message(main_text, small_text)
    line_bot_api.reply_message(event.reply_token, bubble)

def send_preference_update_prompt(event):
    current_value =",".join(user_data["preferences"])
    main_text = f"您目前的飲食偏好為 : {current_value} "
    small_text = "請輸入新增的飲食偏好：(輸入文字即可，EX：起司)"  # 合併字串
    bubble = create_flex_message(main_text, small_text)
    line_bot_api.reply_message(event.reply_token, bubble)

# 建立 Flex Message
def create_flex_message(main_text, example_text):
    bubble = BubbleContainer(
        body=BoxComponent(
            layout="vertical",
            contents=[
                TextComponent(text=main_text, size="md", color="#000000", wrap=True),
                TextComponent(text=example_text, size="sm", color="#888888", wrap=True)
            ]
        )
    )
    return FlexSendMessage(alt_text="更新數值", contents=bubble)

# 處理使用者輸入並更新資料
def process_update(event, text, UserId):
    update_info = user_state.get(UserId)

    if not update_info:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="發生錯誤，請重新嘗試。"))
        return

    update_type = update_info.get("update_type")
    try:
        if update_type == "更新身高":
            handle_height_update(event, UserId, text)
        elif update_type == "更新體重":
            handle_weight_update(event, UserId, text)
        elif update_type == "更新過敏原":
            handle_allergen_update(event, UserId, text)
        elif update_type == "更新飲食偏好":
            handle_preference_update(event, UserId, text)
    except ValueError:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請輸入有效的數字或文字。"))

# 各類型的具體更新處理
def handle_height_update(event, UserId, text):
    new_value = float(text)
    if 50 <= new_value <= 250:
        update_user_value(UserId, "height", new_value)
        reply = f"已成功更新身高為 {new_value} 公分"
    else:
        reply = "您輸入的身高不在合理範圍內，請重新輸入。"
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text=reply))

def handle_weight_update(event, UserId, text):
    new_value = float(text)
    if 20 <= new_value <= 300:
        update_user_value(UserId, "weight", new_value)
        reply = f"已成功更新體重為 {new_value} 公斤"
    else:
        reply = "您輸入的體重不在合理範圍內，請重新輸入。"
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text=reply))

def handle_allergen_update(event, UserId, text):
    user_data["allergens"].add(text)
 # 將完整的過敏原列表顯示
    current_allergens = "\n".join(user_data["allergens"])  # 換行顯示每個過敏原
    main_text = f"已新增過敏原：{text}"  # 顯示新增結果和完整過敏原
    small_text = f"更新後的過敏原為 :\n{current_allergens} "  
    bubble = create_flex_message(main_text, small_text)
    line_bot_api.reply_message(event.reply_token, bubble)

def handle_preference_update(event, UserId, text):
    user_data["preferences"].add(text)
     # 將完整的飲食偏好列表顯示
    current_preferences = "\n".join(user_data["preferences"])  # 換行顯示每個偏好
    main_text
    small_text = f"更新後的飲食偏好為 :\n{current_preferences} "  
    bubble = create_flex_message(main_text, small_text)
    line_bot_api.reply_message(event.reply_token, bubble)

# 更新使用者資料
def update_user_value(UserId, value_type, new_value):
    user_data[value_type] = new_value
    print(f"已更新 {UserId} 的 {value_type} 為 {new_value}")
    
    
# 傳送推薦菜單的模板訊息
def send_recommend_template(reply_token, line_user_id):
    send_recommend_message = TemplateSendMessage(
        alt_text='推薦菜單',
        template=ButtonsTemplate(
            text='您好~\n請問是否要查看推薦菜單?',
            actions=[              
                URIAction(label='查看推薦菜單', uri='https://liff.line.me/2006163998-qrYpNxk9'),  # 每個按鈕都是一個 URIAction 物件
                MessageAction(label='取消', text='取消查看推薦菜單')
            ]
        )
    )

    line_bot_api.reply_message(reply_token, send_recommend_message)

# 傳送查看營養的模板訊息
def send_view_template(reply_token, line_user_id):
    send_view_template = TemplateSendMessage(
        alt_text='飲食日誌',
        template=ButtonsTemplate(
            text='您好~\n請問是否要查看飲食日誌?',
            actions=[              
                URIAction(label='查看飲食日誌', uri='https://liff.line.me/2006163998-2r4nYRNq'),  # 每個按鈕都是一個 URIAction 物件
                MessageAction(label='取消', text='取消查看飲食日誌')
            ]
        )
    )

    line_bot_api.reply_message(reply_token, send_view_template)
    
# 新增紀錄飲食模板
def send_food_template(reply_token, line_user_id):
    send_food_message = TemplateSendMessage(
        alt_text='記錄飲食',
        template=ButtonsTemplate(
            title='記錄飲食',
            text='請選擇您想紀錄的餐別',
            actions=[
                MessageAction(label='早餐', text='記錄早餐'),
                MessageAction(label='午餐', text='記錄午餐'),
                MessageAction(label='晚餐', text='記錄晚餐'),
                MessageAction(label='點心', text='記錄點心')
            ]
        )
    )
    line_bot_api.reply_message(reply_token, send_food_message)

# 處理具體的飲食紀錄輸入
def handle_food_input(event, user_id, meal_type):
    # 抓取使用者記錄時間 (使用事件的時間戳)
    record_time = datetime.fromtimestamp(event.timestamp / 1000)  # event.timestamp 是毫秒，需要除以 1000
    time_str = record_time.strftime('%Y-%m-%d %H:%M:%S')  # 格式化時間為人類可讀形式
    
    # 餐別映射
    meal = {
        "記錄早餐": "早餐",
        "記錄午餐": "午餐",
        "記錄晚餐": "晚餐"
    }.get(meal_type, "點心")
        
    # 定義主要訊息和小字訊息
    main_text = f"您好~\n請問今天的{meal}吃了什麼呢？"
    small_text = "輸入格式為:"
    food_text = "一個麥當勞的滿福堡，一杯麥當勞的咖啡"
    

     # 建立Flex Message
    bubble = BubbleContainer(
        body=BoxComponent(
            layout="vertical",
            contents=[
                TextComponent(text=main_text, size="md", color="#000000"),
                TextComponent(text=small_text, size="sm", color="#888888"),  # 灰色字體，字體變小
                TextComponent(text=food_text, size="sm", color="#888888")  
            ]
        )
    )

    flex_message = FlexSendMessage(alt_text="詢問用餐記錄", contents=bubble)   
    
    # 將用戶狀態和時間存儲在全局的 user_state 中，方便後續處理
    '''user_state[user_id] = {
        "meal_type": meal_type,
        "record_time": time_str  # 存儲紀錄的時間
    }'''
    # 回應訊息
    line_bot_api.reply_message(event.reply_token, flex_message)

    # 如果正在記錄飲食
    if user_id in recording_food:
        recording_food[user_id]['type'] = meal  # 設置 type 為 meal
        TEMP_USERID = user_id
        print("進入後recording_food[UserId]",recording_food)


from linebot.models import TextSendMessage

def handle_food_record(event):
    # 這邊先使用GPT拆解JSON -> 目的:檢查JSON是否有缺漏欄位，有則請使用者補齊
    # 確認都無缺失，再調用[飲食紀錄] record_diet(input_text, openai_api_key, db_settings)
    user_id = event.source.user_id
    input_text = event.message.text
    print("enter handle_food_record",input_text)
    
    # 初始化使用者狀態，若不存在
    if user_id not in user_state:
        print("初始化(前)")
        user_state[user_id] = {"incomplete_items": [], "completed_items": []}
    
    # 檢查是否有未完成的項目正在等待使用者輸入
    if "incomplete_items" in user_state[user_id] and user_state[user_id]["incomplete_items"]:
        incomplete_items = user_state[user_id]["incomplete_items"]
        current_item = incomplete_items[0]  # 取出第一個需要補全的項目

        # 檢查當前項目的缺失欄位
        for field in ["數詞", "量詞", "食物來源"]:
            if not current_item.get(field):
                current_item[field] = input_text  # 使用者輸入的值儲存到 item 中
                break  # 更新一個欄位後跳出循環

        # 檢查此項目是否已完整
        if all(current_item.get(field) for field in ["數詞", "量詞", "食物來源"]):
            #user_state[user_id]["completed_items"].append(current_item)
            
            # 檢查 `completed_items` 是否已包含該項目，若有則覆蓋
            item_index = next((index for (index, d) in enumerate(user_state[user_id]["completed_items"])
                               if d["項目"] == current_item["項目"]), None)
            if item_index is not None:
                # 覆蓋已存在的項目
                user_state[user_id]["completed_items"][item_index] = current_item
            else:
                # 如果不存在，則新增
                user_state[user_id]["completed_items"].append(current_item)
            print("user_state[user_id][completed_items]",user_state[user_id]["completed_items"])
            incomplete_items.pop(0)  # 移除已完成的項目

        # 如果還有未完成的項目，繼續詢問
        if incomplete_items:
            print("(詢問判斷)user_state[user_id][completed_items]",user_state[user_id]["completed_items"])
            next_item = incomplete_items[0]
            for field in ["數詞", "量詞", "食物來源"]:
                if not next_item.get(field):
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text=f"請輸入 '{next_item['食物名稱']}' 的 {field}:")
                    )
                    return  # 暫停此處理，等待使用者輸入

        # 當所有項目都補全後，將完整的項目集合傳遞給 record_diet
        if not incomplete_items:
            
            input_text = json.dumps(user_state[user_id]["completed_items"], ensure_ascii=False)
            input_array = json.loads(input_text)
            

            if isinstance(input_array, list):
                print("檢查到input_array為列表",input_array)
                # 直接將字串組合
                input_array = "，".join(
                    f"{item['數詞']}{item['量詞']}{item['食物來源']}的{item['食物名稱']}"
                    for item in input_array
                )
                print("已組合成字串",input_array)
                
            result_message, nutritional_summary = record_diet(input_array, openai_api_key, db_settings, user_id, recording_food[user_id]['type'])
            print("result_message",result_message)
            print("nutritional_summary",nutritional_summary)
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text=result_message))
            send_nutrition_summary(event.source.user_id, nutritional_summary)
            del user_state[user_id]  # 移除已完成的狀態
            print("end")
            return
    
    # 初始解析使用者輸入
    parsed_items = parse_user_input_with_gpt(input_text)
    for item in parsed_items:
        # 如果項目是完整的，直接加入 completed_items
        if all(item.get(field) for field in ["數詞", "量詞", "食物來源"]):
            user_state[user_id]["completed_items"].append(item)
        else:
            user_state[user_id]["incomplete_items"].append(item)

    # 如果有缺失的項目，詢問使用者補全
    if user_state.get(user_id, {}).get("incomplete_items"):
        first_incomplete = user_state[user_id]["incomplete_items"][0]
        for field in ["數詞", "量詞", "食物來源"]:
            if not first_incomplete.get(field):
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text=f"請輸入 '{first_incomplete['食物名稱']}' 的 {field}:")
                )
                return  # 暫停此處理，等待使用者輸入

    # 如果沒有缺失，直接處理完整輸入
    result_message, nutritional_summary = record_diet(input_text, openai_api_key, db_settings, user_id, recording_food[user_id]['type'])
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text=result_message))
    # send_nutrition_summary(event.source.user_id, nutritional_summary)
    send_nutrition_summary(user_id, nutritional_summary)

def send_nutrition_summary(user_id, nutritional_summary):

    # 使用 datetime.now() 來抓取當前時間
    current_time = datetime.now()
    time_str = current_time.strftime('%Y-%m-%d')  # 格式化時間
    
    # 假設每餐的營養數據是根據輸入食物進行動態計算的，這裡是示例數值
    bubble = BubbleContainer(
        body=BoxComponent(
            layout="vertical",
            contents=[
                TextComponent(text=f"今日{recording_food[user_id]['type']}營養總計", weight="bold", size="xl", margin="md"),
                TextComponent(text=time_str, size="sm", color="#aaaaaa", margin="md"),

                # 分隔線
                SeparatorComponent(margin="md"),
                BoxComponent(
                    layout="horizontal",
                    contents=[
                        TextComponent(text="卡路里", size="sm", color="#555555", flex=2),
                        TextComponent(text=f"{round(nutritional_summary['熱量'], 2)} kcal", size="sm", color="#111111", flex=1)
                    ]
                ),
                BoxComponent(
                    layout="horizontal",
                    contents=[
                        TextComponent(text="蛋白質", size="sm", color="#555555", flex=2),
                        TextComponent(text=f"{round(nutritional_summary["蛋白質"], 2)} g", size="sm", color="#111111", flex=1)
                    ]
                ),
                BoxComponent(
                    layout="horizontal",
                    contents=[
                        TextComponent(text="脂質", size="sm", color="#555555", flex=2),
                        TextComponent(text=f"{round(nutritional_summary["脂質"], 2)} g", size="sm", color="#111111", flex=1)
                    ]
                ),
                BoxComponent(
                    layout="horizontal",
                    contents=[
                        TextComponent(text="醣類", size="sm", color="#555555", flex=2),
                        TextComponent(text=f"{round(nutritional_summary["醣類"], 2)} g", size="sm", color="#111111", flex=1)
                    ]
                ),
                SeparatorComponent(margin="md"),
                TextComponent(text="以上為估算值，請參考食物包裝或營養資訊。", size="xs", color="#aaaaaa", wrap=True)
            ]
        )
    )
    # 完成記錄後將狀態重置
    recording_food[user_id] = {"recording": False}
    flex_message = FlexSendMessage(alt_text="營養總計", contents=bubble)
    line_bot_api.push_message(user_id, flex_message)
    

    
# 啟動 Flask 應用
if __name__ == "__main__":
    app.run(port=port)


In [ ]:
# ngrok隧道占滿如何解決： STEP1 快速刪除全部佔用空間
from pyngrok import ngrok

# 列出當前運行的隧道
tunnels = ngrok.get_tunnels()
print("Current running tunnels:")
for tunnel in tunnels:
    print(tunnel.public_url)

# =============================
# 停止所有運行中的隧道
ngrok.kill()

# 確認所有隧道已停止
print("All tunnels have been stopped.")


In [ ]:
# ngrok隧道占滿如何解決： STEP2 查詢是否全部刪除
from pyngrok import ngrok

# 列出當前運行的隧道
tunnels = ngrok.get_tunnels()
print("Current running tunnels:")
for tunnel in tunnels:
    print(tunnel.public_url)